# 07-Das Grundprinzip von Gruppieren
Der Prozess des Gruppieren ("group by") bezieht sich auf ein oder mehrere Handlung:
* **Splitting** Also das Zerlegen des Datensatzes in bestimmte Gruppen
* **Applying** Anwenden einer Funktion auf jede dieser Gruppen
* **Combining** Zusammenfassen der Daten in eine neue Datenstruktur

Das **Splitting** ist der zentrale Schritt und wird oft als erstes ausgeführt. Meisten sollen die Datensätze in den Gruppen aber noch weiterverarbeitet werden. Daher stehenim Schritt **Applying** unterem anderem die folgenden Funktionen zur Verfügung:

* **Aggregation:** Kummulieren der Daten mit Statistischen Methoden, wie zum Beispiel:
    * Summen und Mittelwerte
    * Zählen der Werte
    
    
* **Transformation:**
    * Standardisierung der Daten innerhalb einer Gruppe
    * Aufüllen von fehlenden Daten innerhalb einrer Gruppe mit aus der Gruppe abgeleitetend Werte

* **Filtration:**
    * Gruppenbezogenes Filtern, z.B. das Aussortieren von Ausreißern innerhalb einer Gruppe
    

* Oder aber die Kombination verschiedener Schritte


---
*Übersetz aus folgender Quelle: https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html*



# Arbeiten mit ``.groupby()``
## Datageneration

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Generieren von Übungsdaten
df_students = pd.DataFrame({'student_id':list(range(100,107)), 
                            'subject_id':[1,2,3,1,2,3,1],
                            'subject': ['Math', 'Geo', 'Business', 'Math', 'Geo', 'Business' ,'Math'],
                            'age':[18,22,31,22,26,21,20],
                            'left_hand':[False,True,False,True,False,False,True]})
df_students

## Zählen von Datensätzen ``.groupby().count(<column>)``

In [ ]:
# Mit groupby.count() können wir uns anzeigen lassen, wie viel Datensätze jede Gruppe besitzt 
df_students.groupby('subject').count()

In [ ]:
# Alternativ können wir auch nach der Größe fragen
df_students.groupby('subject').size()

**Unterschied zwischen ``.count()`` und ``.size()``**

In [ ]:
# Unterschied zwischen .count() und .size()
df_students_nan = pd.DataFrame({'student_id':list(range(100,107)), 
                            'subject_id':[1,2,3,1,2,np.NaN,1],
                            'subject': ['Math', 'Geo', 'Business', 'Math', 'Geo', 'Business' ,'Math'],
                            'age':[np.NaN,22,31,22,26,21,20],
                           'left_hand':[False,True,False,True,False,False,True]})
df_students_nan

In [ ]:
# count wird durch NaN Werte beeinflusst
df_students_nan.groupby('subject').count()

In [ ]:
# Size verhindert diesen Effekt
df_students_nan.groupby('subject').size()

## Schlüssel/Index des Groupby-Objekt

In [ ]:
# Da wir nach Subject gruppieren, werden die einzigeartigen Werte der Spalte "subject" der neue Index. 
# Den enstehenden Index können wir auch generieren wenn wir uns die einzigartigen Werte der Spalte "subject" anschauen
df_students['subject'].unique()

## Aggregation von Daten

Wichtigste Funktionen für die Aggregation:
* ``.sum()`` - *Summe*
* ``.mean()`` - *Mittelwert*
* ``.min()`` - *Minimalwert*
* ``.max()`` - *Maximalwert*

In [ ]:
# Bilden der Summe nach der "subject_id", text wird ignoriert
df_students.groupby('subject_id').sum()

In [ ]:
# Durchschnittsalter berechnen
df_students[['subject','age']].groupby('subject').mean()

## Auszug aus dem Dataframe betrachten
Mit ``.first()`` und ``.last()`` bekommen wir die einen neuen Dataframe zurück in dem nur der erste oder der letzte Datensatz einer jeden Gruppe enthalten ist. Dies kann besonders nützlich sein, wenn die Datensätze in einer bestimmen Reihenfolge im Datensatz auftauchen.

In [ ]:
df_students.groupby('subject').first()

In [ ]:
df_students.groupby('subject').first()

**Auch ``.head()`` und ``.tail()`` lassen sich hier wie gewohnt verwenden**

In [ ]:
df_students.groupby('subject').head(2)

## Mehrere Gruppierungsebenen
Oft bleibt es nicht bei einer Gruppierungsebene. Wie können beliebig viele Gruppierungsebenen hinzufügen. Die Reihenfolge hat zunächst keinen Erkennbaren Einfluss auf das Endergebnis, außer, dass sich die Sortierung der Daten ändert.

In [ ]:
# Größeres Datenset einlesen
df = pd.read_csv('../src/bigdata/120-years-of-olympic-history-athletes-and-results/athlete_events.csv')

In [ ]:
# Auswählen von ein paar wahren Champions
df = df[df['ID'].isin([69210, 107383, 127501])]
df

In [ ]:
# Uns interessieren die Medallien und zwar nur die Gewinne
df['Medal'].unique()

**Umgang mit fehlenden oder fehlerhaften Werten, mit ``.dropna()`` und ``.fillna()``**

In [ ]:
# Eine Möglichkeit ist die Daten zu verwerfen
df_drop = df.dropna(subset=['Medal'])
print(len(df))
print(len(df_drop))

In [ ]:
# Bessser jedoch ist, wenn wir den fehlenden Daten eine Bedeutung und einen Wert zukommen lassen
df['Medal'] = df['Medal'].fillna('Teilnahme')
df.sample(10)

In [ ]:
# Teilnehmer ist einzigartig zugeordnet über die ID. Sie bildet das oberste Level.
df.groupby(['ID', 'Name', 'Team', 'Medal']).size()

In [ ]:
df.groupby(['Medal', 'ID', 'Name', 'Team']).size()

**Gruppierung zurücksetzen und in "normalen" DataFrame zurückverwandeln mit ``.reset_index()`` und umbennen von Spalten mit ``.rename()``**  

In [ ]:
# Zurück wandeln eines "grouped" DataFrames mit .reset_index() 
df_norm = df.groupby(['ID', 'Name', 'Team', 'Medal']).size().reset_index()
df_norm = df_norm.rename(columns={0:'Anzahl'})
df_norm.sort_values(['Name', 'Anzahl'], ascending=[True, False])

# Tipps and Tricks

###  Referenz aller Funktionen des groupby-Objekt
* https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#computations-descriptive-stats

### Vollständige Tutorial zu Gruppieren aus der offiziellen Pandas Dokumentation
* https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
* https://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html#cookbook-grouping
    
